# CAI Lab Session 2: Intro to ElasticSearch

In this session you will learn:

- a few basics of the `ElasticSearch` database
- how to index a set of documents and how to ask simple queries about these documents
- how to do this from `Python`
- based on the previous, you will compute the boolean and tf-idf matrix for the toy corpus used in class

## 1. ElasticSearch

[ElasticSearch](https://www.elastic.co/) is a _NoSQL/document_ database with the capability of indexing and searching text documents. As a rough analogue, we can use the following table for the equivalence between ElasticSearch and a more classical relational database:

| Relational DB | ElasticSearch |
|---|---|
| Database | Index |
| Table | Type |
| Row / record | Document |
| Column | Field |

An index can be thought of as an optimized collection of documents and each document is a collection of fields, which are the key-value pairs that contain your data.

`ElasticSearch` is a pretty big beast with many options. Luckily, there is much documentation, a few useful links are:

- Here is the [full documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)
- Intros you may want to have a look at: 
    - https://medium.com/expedia-group-tech/getting-started-with-elastic-search-6af62d7df8dd
    - http://joelabrahamsson.com/elasticsearch-101
- You found another one that you liked? Let us know. 

## 2. Running ElasticSearch

First you will need to install `ElasticSearch` following instructions in their [documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html).

This database runs as a web service in a machine and can be accessed using a REST web API; however we will interact with the database through its python libraries `elasticsearch-py` and `elasticsearch-dsl`, so you will need to install these as well.  You can run `ElasticSearch` by typing from the command-line prompt:

```
$ <path_to_elasticsearch_bin>/elasticsearch &
```



After a few seconds (and a lot of logging) the database will be up and running; you may need to hit return for the prompt to show up. To test whether `ElasticSearch` is working execute the code in the cell below. __The database needs to be running throughout the execution of this script, otherwise you will get a connection error.__

In [52]:
from pprint import pprint

In [53]:
import requests

try:
    resp = requests.get('http://localhost:9200/')
    pprint(resp.content)
    
except Exception:
    print('elasticsearch is not running')

(b'{\n  "name" : "DESKTOP-QNOM0EQ",\n  "cluster_name" : "elasticsearch",\n  "c'
 b'luster_uuid" : "eig_fG5ySRWEscM_Y6LUcQ",\n  "version" : {\n    "number" : '
 b'"8.10.2",\n    "build_flavor" : "default",\n    "build_type" : "zip",\n    '
 b'"build_hash" : "6d20dd8ce62365be9b1aca96427de4622e970e9e",\n    "build_da'
 b'te" : "2023-09-19T08:16:24.564900370Z",\n    "build_snapshot" : false,\n  '
 b'  "lucene_version" : "9.7.0",\n    "minimum_wire_compatibility_version" :'
 b' "7.17.0",\n    "minimum_index_compatibility_version" : "7.0.0"\n  },\n  "t'
 b'agline" : "You Know, for Search"\n}\n')


If `ElasticSearch` is working you will see an answer from the server; otherwise you will see a message indicating that it is not running. You can try also throwing the URL http://localhost:9200 to your browser; you should get a similar answer.

**In version 8 they introduced enhanced security, which may give you trouble when executing the code here, to deal with this you can either install an earlier version (7 or older) or turn off security settings in their `config/elasticsearch.yml` config file (just set to _false_ everything concerning the security options).** Since we are using the database in offline, local mode this should not be a problem.

Also, you should run this script locally in your machine, if you use Google Collab or similar this is not going to work because elasticsearch should be running on the machine where the script is being executed.

## 3. Indexing and querying

`ElasticSearch` is a database that allows storing documents (tables do not need a predefined schema as in relational databases). Text in these documents can be processed so the queries extend beyond exact matches allowing complex queries, fuzzy matching and ranking documents respect to the actual match. 

These kinds of databases are behind search engines like Google Search or Bing.

There are different ways of operating with ElasticSearch. It is deployed esentially as a web service with a REST API, so it can be accessed basically from any language with a library for operating with HTTP servers.

We are going to use two python libraries for programming on top of ElasticSearch: `elasticsearch` and `elasticsearch-dsl`. Both provide access to ElasticSearch functionalities hiding and making more programming-friendly the interactions, the second one is more convenient for configurating and searching. Make sure both python libraries are installed to proceed with this session.

We are only going to see the essential elements for developing the session but feel free to learn more.

To interact with ElasticSearch with need a client object of type `Elasticsearch`.

In [54]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://localhost:9200", timeout=1000)

C:\Users\Carlos Arbonés\AppData\Local\Temp\ipykernel_4800\2120730465.py:3: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  client = Elasticsearch("http://localhost:9200", timeout=1000)


With this client you have a connection for operating with Elastic search. Now we will create an index. There are index operations in each library, but the one in `elasticseach-dsl` is simpler to use.

In [55]:
from elasticsearch_dsl import Index

index = Index('test', using=client)  # if it does not exist, it is created; if it does exist, then it connects

First we will need some text to index, for testing purposes we are going to use the python library `loremipsum`. We will need to install it first if it is not installed already, uncomment the code in next cell if you need to install the library

Now we create some random paragraphs

In [56]:
import lorem

texts = [lorem.paragraph() for _ in range(10)]
print(len(texts))
pprint(texts[0])

10
('Dolor aliquam modi quaerat porro quaerat porro. Consectetur ipsum voluptatem '
 'amet etincidunt magnam eius eius. Magnam aliquam est porro labore sed amet. '
 'Aliquam etincidunt adipisci velit porro consectetur velit. Ipsum dolor sit '
 'labore aliquam. Consectetur quisquam est amet. Ipsum magnam numquam tempora '
 'non tempora porro.')


Now we can index the paragraphs in ElasticSearch using the `index` method. The document is passed as a python dictionary with the `document` parameter. The keys of the dictionary will be the fields of the document, in this case we well have only one (`text`) -- here, we use this tag but could use anything we wanted to.

In [57]:
for t in texts:
    client.index(index='test', document={'text': t})
    print(f'Indexing new text: {t[:70]} ...')

Indexing new text: Dolor aliquam modi quaerat porro quaerat porro. Consectetur ipsum volu ...
Indexing new text: Tempora porro magnam adipisci quisquam amet consectetur. Quaerat conse ...
Indexing new text: Numquam eius sed porro amet dolor amet. Neque dolor tempora dolore dol ...
Indexing new text: Dolor ut quaerat dolorem non quiquia voluptatem amet. Ipsum magnam mag ...
Indexing new text: Neque ut eius voluptatem est. Dolor dolorem dolorem labore voluptatem  ...
Indexing new text: Quaerat modi quaerat amet quisquam adipisci magnam dolore. Quaerat tem ...
Indexing new text: Neque amet dolore modi tempora. Etincidunt numquam labore est. Amet co ...
Indexing new text: Sed modi est consectetur neque quisquam consectetur non. Eius ipsum ve ...
Indexing new text: Adipisci ut modi dolor ipsum consectetur. Tempora amet tempora neque.  ...
Indexing new text: Eius neque modi neque dolore modi. Neque eius velit quiquia ut. Dolore ...


In case we want to get all docs in the index, we can do the following:

In [58]:
# get all docs in index 'test'
resp = client.search(index="test", query={"match_all": {}})

# print them
print(f"Got {resp['hits']['total']['value']} hits:")
for hit in resp['hits']['hits']:
    pprint(hit["_source"])

Got 50 hits:
{'text': 'Magnam etincidunt ipsum voluptatem ipsum quaerat dolorem. Numquam '
         'dolor eius aliquam porro non. Dolore tempora numquam velit amet est '
         'dolor dolore. Quiquia adipisci aliquam ut non ut. Numquam neque '
         'etincidunt etincidunt adipisci. Dolore tempora ipsum quisquam amet '
         'neque. Magnam numquam amet voluptatem. Consectetur quiquia non '
         'consectetur ut velit modi. Dolorem ut ipsum dolor.'}
{'text': 'Ut dolore sed etincidunt. Magnam adipisci adipisci magnam dolor '
         'porro. Etincidunt consectetur porro quaerat sit neque dolore '
         'consectetur. Sed neque consectetur amet ut. Sit dolore modi labore '
         'sed.'}
{'text': 'Quaerat non eius adipisci labore. Labore ipsum modi numquam ut. '
         'Adipisci amet ipsum tempora dolorem porro. Ut amet voluptatem magnam '
         'consectetur non. Sed consectetur adipisci eius ipsum dolore est. '
         'Eius dolore ipsum dolor ut quaerat dolore. Quis

We can also search for documents that contain a given keyword:

In [59]:
from elasticsearch_dsl import Search

# the following search query specifies the field where we want to search
s_obj = Search(using=client, index='test')
sq = s_obj.query('match', text='non')
resp = sq.execute()

print(f'Found {len(resp)} matches.')

for hit in resp:
    print(f'\nID: {hit.meta.id}\nText: {hit.text}')

Found 10 matches.

ID: 47DF74oBTWYgklMH0QF5
Text: Non ut non dolor. Quiquia ipsum quiquia quaerat quisquam. Sit sed labore ut modi consectetur quiquia. Velit modi tempora dolorem dolorem magnam sed porro. Etincidunt eius quiquia non est aliquam ipsum. Dolor tempora neque ut dolor magnam. Etincidunt est aliquam quisquam neque non adipisci velit. Neque labore modi sit non est dolorem non.

ID: -bDI74oBTWYgklMHbgEM
Text: Non etincidunt dolore dolor. Non porro quisquam quiquia. Consectetur eius eius etincidunt dolor. Dolorem eius non dolorem numquam dolor. Porro non consectetur voluptatem modi dolor. Etincidunt eius consectetur modi quisquam magnam eius tempora. Quiquia etincidunt magnam aliquam neque consectetur quiquia dolore.

ID: 37DF74oBTWYgklMH0QE4
Text: Numquam voluptatem numquam dolorem. Dolor labore consectetur etincidunt. Sit quisquam non dolor amet adipisci. Eius ut est aliquam voluptatem consectetur. Quiquia voluptatem non etincidunt labore non dolorem. Ut dolorem modi est aliq

## 4. Counting words and docs

`Elastic search` helps us to obtain the counts of words in each document. For example, the following code obtains the counts of words of a whole index by adding the counts of words obtained from each document through the functionality of `termvectors`. This function also allows us to get _document counts_ for computing tf-idf weights, by setting the `term_statistics` option to `True`.

In [60]:
from elasticsearch.helpers import scan
from collections import Counter

# Search for all the documents and query the list of (word, frequency) of each one
# Totals are accumulated using a Counter for term frequencies, but not for document freqs.
word_counts = Counter()
doc_counts = Counter()
sc = scan(client, index='test', query={"query" : {"match_all": {}}})
for s in sc:
    tv = client.termvectors(index='test', id=s['_id'], fields=['text'], term_statistics=True, positions=False)
    if 'text' in tv['term_vectors']:   # just in case some document has no field named 'text'
        for t in tv['term_vectors']['text']['terms']:
            word = t
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            #pprint(tv['term_vectors']['text']['terms'][t])
            word_counts.update({word: count})  # número de veces que aparece la palabra en el documento
            doc_counts[word] = df      # número de documentos en los que aparece la palabra

In [61]:
# show word frequencies
word_counts.most_common()

[('adipisci', 115),
 ('consectetur', 110),
 ('quaerat', 106),
 ('ut', 102),
 ('quisquam', 98),
 ('magnam', 96),
 ('neque', 92),
 ('dolore', 91),
 ('dolor', 90),
 ('dolorem', 90),
 ('tempora', 90),
 ('amet', 87),
 ('eius', 87),
 ('quiquia', 87),
 ('porro', 86),
 ('velit', 86),
 ('numquam', 85),
 ('modi', 83),
 ('non', 81),
 ('labore', 81),
 ('sed', 80),
 ('ipsum', 79),
 ('aliquam', 78),
 ('sit', 78),
 ('est', 76),
 ('voluptatem', 76),
 ('etincidunt', 72)]

In [62]:
# show doc freq
doc_counts.most_common()

[('quaerat', 46),
 ('adipisci', 45),
 ('tempora', 45),
 ('consectetur', 44),
 ('modi', 44),
 ('non', 44),
 ('quisquam', 43),
 ('dolorem', 42),
 ('neque', 42),
 ('eius', 41),
 ('ut', 41),
 ('labore', 41),
 ('amet', 40),
 ('dolor', 40),
 ('porro', 40),
 ('velit', 40),
 ('sit', 40),
 ('aliquam', 39),
 ('magnam', 39),
 ('numquam', 39),
 ('quiquia', 39),
 ('voluptatem', 39),
 ('dolore', 38),
 ('ipsum', 38),
 ('est', 37),
 ('etincidunt', 37),
 ('sed', 34)]

## 5. Proposed simple exercise

To get more familiar with elasticsearch, we propose that you _generate the Boolean and tf-idf matrices_ for the toy example that we used in class. You will find 7 text documents that contain the toy documents with the materials for this session in the racó. The steps to follow are:

- create an empty index
- open each text document in the `toy-docs` folder provided, read its contents and add it to the index as a new document; your index should contain 7 documents after this
- use the `termvectors` function to obtain term and doc counts, generate Boolean and tf-idf matrices based on these counts
- double check that your results coincide with the numbers in theory slides

In [63]:
from elasticsearch_dsl import Index

index = Index('exercise', using=client)

In [64]:
for i in range(1, 8):
    with open('toy-docs/' + f'd{i}.txt', 'r') as f:
        text = f.read()
        client.index(index='exercise', document={'text': text})

In [65]:
resp = client.search(index="exercise", query={"match_all": {}})
print(f"Got {resp['hits']['total']['value']} hits")

Got 0 hits


In [77]:
import numpy as np

boolean_matrix = np.zeros((7, 6), dtype=int)
vector_matrix = np.zeros((7, 6))
map = {'five': 0, 'four': 1, 'one': 2, 'six': 3, 'three': 4, 'two': 5}	

word_counts = Counter()
doc_counts = Counter()
sc = scan(client, index='exercise', query={"query" : {"match_all": {}}})
for i, s in enumerate(sc):
    tv = client.termvectors(index='exercise', 
                            id=s['_id'], 
                            fields=['text'], 
                            term_statistics=True, 
                            positions=False)
    
    if 'text' in tv['term_vectors']:   
        for t in tv['term_vectors']['text']['terms']:
            word = t
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            word_counts.update({word: count})  
            doc_counts[word] = df  
            
            boolean_matrix[i][map[word]] = 1
            vector_matrix[i][map[word]] = count/max(word_counts.values())*np.log(7/df)
            
pprint(boolean_matrix) 
pprint(vector_matrix)

array([[0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 1],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0]])
array([[0.        , 0.        , 0.84729786, 0.        , 0.15415068,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.07707534,
        1.25276297],
       [1.25276297, 0.28243262, 0.28243262, 0.        , 0.05138356,
        0.        ],
       [0.        , 0.        , 0.28243262, 0.56486524, 0.03853767,
        0.83517531],
       [0.        , 0.42364893, 0.        , 0.14121631, 0.02569178,
        0.        ],
       [0.        , 0.        , 0.        , 0.28243262, 0.0578065 ,
        0.        ],
       [0.15659537, 0.10591223, 0.        , 0.        , 0.        ,
        0.        ]])


## 6. Cleanup

Finally, we remove the test index..

In [67]:
#index.delete()